# SPAM


In [159]:
import sqlite3
import requests  # To get the pages
import hashlib
import re

# from bs4 import BeautifulSoup # and to process them

con = sqlite3.connect("spam.db")
cur = con.cursor()


## Globals

In [87]:
base_url = "http://web.mit.edu/jync/www/spam/"
extension = ".html"

## Scraper Table

In [144]:
st = cur.execute("CREATE TABLE IF NOT EXISTS \"url_cache\" (\
	\"hash\"	TEXT,\
	\"text\"	TEXT\
);")


## Poem Table

In [152]:
# pt = cur.execute("DROP TABLE poems;")

pt = cur.execute("CREATE TABLE IF NOT EXISTS \"poems\" (\
	\"id\"	INTEGER UNIQUE,\
	\"line_1\"	TEXT,\
	\"line_2\"	TEXT,\
	\"line_3\"	TEXT,\
	\"author\"	TEXT\
);")
print(pt)

# Scraper

In [146]:
for i in range ( 1001,19601,100 ) :
    url = base_url + str(i) + "-" + str(i + 99) + extension
    # print(url)
    
    ID = i
    hash = hashlib.md5(url.encode()).hexdigest()
    
    # https://stackoverflow.com/a/9756276
    res = cur.execute("SELECT EXISTS(SELECT 1 FROM url_cache WHERE hash='"+hash+"');").fetchone()[0]
    
    if res is 0 :
        r = requests.get(url)
        text = r.text
        cur.execute("INSERT INTO url_cache VALUES ( ?, ?)", ( hash, text ) )
        # print(hash)

    else :
        # print(hash)
        continue 
        
con.commit()

In [164]:
for i in range ( 1001,19601,100 ) :
    url = base_url + str(i) + "-" + str(i + 99) + extension
    # print(url)
    
    ID = i
    hash = hashlib.md5(url.encode()).hexdigest()
    # print(hash)
    res = cur.execute("SELECT text FROM url_cache WHERE `hash` = '" + hash + "'" )
    text = res.fetchone()[0]
    
    # print(len(text))
    matches = re.findall(r"<SPAN .*\n.*\n.*\n.*\n.*\n.*\n.*<P>",text)
    
    for match in matches :
        # Found some matches
        # print( match )
        groups = re.findall(r"<SPAN CLASS='Number'>(.*)\.</SPAN><BR>\n.*\n(.*)<BR>\n(.*)<BR>\n(.*)<BR>\n.*\n.*>--(.*)</ADDRESS><P>", match)[0]
        
        if len( groups) == 5 :
            # https://stackoverflow.com/a/4869782

            poem_values = [ int(groups[0]), 
                           re.sub('<[^>]*>', '', groups[1]), 
                           re.sub('<[^>]*>', '', groups[2]),
                           re.sub('<[^>]*>', '', groups[3]),
                           re.sub('<[^>]*>', '', groups[4]),]
            # print(poem_values)

            insert_results = cur.execute("INSERT OR REPLACE INTO poems VALUES ( ?, ?, ?, ?, ? )", poem_values )
            # print(insert_results)
        else :
            print(len(groups))
    # break

con.commit()

22715
22918
23262
22592
22313
22383
22952
23193
23488
23865
23524
23554
22796
23392
23937
23182
22796
23553
22423
21890
23073
23199
24286
23245
22693
23730
24089
24290
23632
22840
24922
24766
23707
24572
23676
23528
22920
23640
23604
23936
24833
23080
22766
22821
22798
22198
22351
22446
22146
22273
23075
21743
22750
23239
22397
21682
22539
22085
22022
21642
22195
22685
22788
22559
21473
22909
22981
22548
22798
22458
22250
21442
23115
22321
21583
22575
23948
22641
21779
21630
21258
21757
21882
22726
21888
22395
22458
22629
22541
21894
22027
22566
22463
22248
22187
23601
23391
22994
22598
22454
22932
22390
22610
22158
22264
22821
22670
22600
23091
22387
22151
22740
22376
22471
21933
22258
22857
22889
22551
22488
22149
22870
22265
22734
23010
22411
22363
22564
22208
21719
22017
22431
21875
22509
22684
22192
22363
22176
22021
21655
21619
22513
22703
22325
22535
22291
21858
22308
21916
22595
22619
21925
22284
22648
22378
22318
22493
22048
22654
21600
22113
22193
21887
21546
21570
21475
2168

In [157]:
con.close()